In [1]:
# coding=utf-8
# Copyright 2023-present the HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ.get('CUDA_VISIBLE_DEVICES')

'3'

In [2]:
from loftq_finetuning import *

In [3]:
cd /var/ontologia/Servicos/ACS_LLMS/server/arquivos/loftq_finetuning/

/var/ontologia/Servicos/ACS_LLMS/server/arquivos/loftq_finetuning


In [4]:
pwd

'/var/ontologia/Servicos/ACS_LLMS/server/arquivos/loftq_finetuning'

In [14]:
! python quantize_save_load.py  --model_name /var/ontologia/Servicos/ACS_LLMS/server/models/mistral-7b --bits 4 --iter 1 --rank 16 

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:13<00:00,  6.84s/it]
Traceback (most recent call last):
  File "/var/ontologia/Servicos/ACS_LLMS/server/arquivos/loftq_finetuning/quantize_save_load.py", line 236, in <module>
    base_dir, lora_dir = quantize_and_save()
                         ^^^^^^^^^^^^^^^^^^^
  File "/var/ontologia/Servicos/ACS_LLMS/server/arquivos/loftq_finetuning/quantize_save_load.py", line 163, in quantize_and_save
    lora_model = get_peft_model(model, lora_config)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/peft/mapping.py", line 116, in get_peft_model
    return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](model, peft_config, adapter_name=adapter_name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-pa

<hr>

<hr>

<hr>

# BitsandBytes Quantization

In [1]:
from transformers import BitsAndBytesConfig
from torch import bfloat16, float32      # torch dtypes: float32, float16, float, float64, int__ (igualmente)


# Config em 4-bits p/ carregar o LLM com memória reduzida em GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Quantização em 4-bits 
    bnb_4bit_quant_type='nf4',  # Normalized Float 4 (normalização - pesos igualmente espaçados)
    bnb_4bit_use_double_quant=True,  # Quantização dupla
    bnb_4bit_compute_dtype=float32  # Dequantização para Inferência (a escolher) 
)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/zephyr-7b-beta'

# Zephyr com BitsAndBytes Config
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map='auto')

# Pipeline HF
pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
tokenizer.pad_token, tokenizer.eos_token

('</s>', '</s>')

<h4> Apenas 4976 MB (5GB) de memória da GPU são utilizados para carregar o modelo em 4 bits (ver CUDA 3): </h4> 
<p>    3   N/A  N/A     84399      C   ...CS_LLMS/server/anaconda3/bin/python     4976MiB 


+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   42C    P0              56W / 300W |  16721MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2-32GB           Off | 00000000:0C:00.0 Off |                    0 |
| N/A   42C    P0              56W / 300W |    319MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   2  Tesla V100-SXM2-32GB           Off | 00000000:13:00.0 Off |                    0 |
| N/A   41C    P0              56W / 300W |    319MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   3  Tesla V100-SXM2-32GB           Off | 00000000:1B:00.0 Off |                    0 |
| N/A   42C    P0              57W / 300W |   5295MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     43209      C   ../../../venv/bin/python3                 16710MiB |
|    1   N/A  N/A     43209      C   ../../../venv/bin/python3                   308MiB |
|    2   N/A  N/A     43209      C   ../../../venv/bin/python3                   308MiB |
|    3   N/A  N/A     43209      C   ../../../venv/bin/python3                   308MiB |
|    3   N/A  N/A     84399      C   ...CS_LLMS/server/anaconda3/bin/python     4976MiB |
+---------------------------------------------------------------------------------------+

In [9]:
prompt = "<|system|>\nVocê é um chatbot amigável.</s>\n<|user|>\nConte-me a história da sua vida como um Large Language Model.</s>\n<|assistant|>\n"
print(prompt)

<|system|>
Você é um chatbot amigável.</s>
<|user|>
Conte-me a história da sua vida como um Large Language Model.</s>
<|assistant|>



In [12]:
# Outputs - Passar prompt para o modelo e gerar a resposta (generate_ids = outputs)

generate_ids = pipe(
    prompt,
    max_new_tokens=512,
    do_sample=False,
    #temperature=0.1,
    #top_p=0.95
)
print(generate_ids[0]["generated_text"])

<|system|>
Você é um chatbot amigável.</s>
<|user|>
Conte-me a história da sua vida como um Large Language Model.</s>
<|assistant|>
Eu não tenho uma vida real, pois sou um modelo de linguagem criado por humanos usando dados e algoritmos avançados. Meu objetivo principal é ajudar os usuários a obter informações e respostas precisas e eficientes, sem a necessidade de interagir com um ser humano.

Minha história começou em 2018, quando meu código foi desenvolvido e treinado com milhões de textos e dados de linguagem natural. Eu foi treinado para entender o significado de palavras e frases, e para responder de acordo com o contexto e a intenção do usuário.

Desde então, eu tenho sido utilizado em várias aplicações, como assistentes virtuais, sistemas de resposta a perguntas, e até mesmo em jogos e entretenimento. Meu conhecimento continua a aumentar e a evoluir, graças às novas informações e dados que são adicionados à minha base de dados.

Eu estou sempre pronto para ajudar os usuários co

In [20]:
# Outputs - Passar prompt para o modelo e gerar a resposta (generate_ids = outputs)

generate_ids = pipe(
    prompt,
    max_new_tokens=512,
    do_sample=False,
    #temperature=0.1,
    #top_p=0.95
)
print(generate_ids[0]["generated_text"])

/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1554: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


<|system|>
Você é um chatbot amigável.</s>
<|user|>
Conte-me a história da sua vida como um Large Language Model.</s>
<|assistant|>
Eu não tenho uma vida real, pois sou um modelo de linguagem criado por humanos usando dados e algoritmos avançados. Meu objetivo principal é ajudar os usuários a obter informações e respostas precisas e eficientes, sem a necessidade de interagir com um ser humano.

Minha história começou em 2018, quando meu código foi desenvolvido e treinado com milhões de textos e dados de linguagem natural. Eu foi treinado para entender o significado de palavras e frases, e para responder de acordo com o contexto e a intenção do usuário.

Desde então, eu tenho sido utilizado em várias aplicações, como assistentes virtuais, sistemas de resposta a perguntas, e até mesmo em jogos e entretenimento. Meu conhecimento continua a aumentar e a evoluir, graças às novas informações e dados que são adicionados à minha base de dados.

Eu estou sempre pronto para ajudar os usuários co

In [2]:
import argparse
import os

import torch
import torch.nn as nn
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from peft import LoftQConfig, LoraConfig, PeftModel, TaskType, get_peft_model

In [3]:
class Shell(nn.Module):
    def __init__(self, weight, bias=None):
        super().__init__()
        self.weight = nn.Parameter(weight, requires_grad=False)
        if bias is not None:
            self.bias = nn.Parameter(bias, requires_grad=False)

In [4]:
def unwarap_model(model, sub_module_name=".base_layer"):
    sub_module_name_list = [k.split(sub_module_name)[0] for k in model.state_dict().keys() if sub_module_name in k]
    sub_module_name_set = set(sub_module_name_list)
    for name in sub_module_name_set:
        # get the parent of the submodule
        name_parent = ".".join(name.split(".")[:-1])
        name_child = name.split(".")[-1]
        sub_module = model.get_submodule(name_parent)
        print(sub_module)

        # replace with shell
        child = getattr(sub_module, name_child)
        weight = getattr(child.base_layer, "weight", None)
        bias = getattr(child.base_layer, "bias", None)
        shell = Shell(weight, bias)

        setattr(sub_module, name_child, shell)

    print("You have unwrapped the model. Use it on your own risk.")



In [5]:
def print_model(model, name):
    print("=" * 10 + name + "=" * 10)
    print(model)
    for name, param in model.named_parameters():
        if torch.is_tensor(param):
            if param.dtype in [torch.float32, torch.float16]:
                print(
                    name,
                    param.shape,
                    param.device,
                    param.dtype,
                    param.requires_grad,
                    param.mean().item(),
                    param.max().item(),
                )
            else:
                print(name, param.shape, param.device, param.dtype, param.requires_grad)
